In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import glob
import matplotlib.pyplot as plt
import pickle
from moviepy.editor import VideoFileClip
from google.colab.patches import cv2_imshow
%matplotlib inline


  if event.key is 'enter':



In [ ]:
import numpy as np
import cv2
import glob
import pickle
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import torch
import torchvision
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Install YOLOv5 if not already installed
import subprocess
import sys
import os

def install_yolov5():
    """Install YOLOv5 dependencies"""
    try:
        # Try to install ultralytics first (more reliable)
        subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics", "-q"])
        print("Ultralytics installed successfully")
    except:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "yolov5", "-q"])
            print("YOLOv5 installed successfully")
        except:
            print("Failed to install YOLOv5 packages")

    try:
        # Try custom model first
        if os.path.exists('best.pt'):
            print("Loading custom YOLOv5 model...")
            model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True, trust_repo=True)
        else:
            print("Custom model not found, using YOLOv5s pretrained...")
            model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, trust_repo=True)
    except Exception as e:
        print(f"Error loading model from hub: {e}")
        print("Trying alternative approach...")
        try:
            # Alternative: use ultralytics YOLO
            from ultralytics import YOLO
            model = YOLO('yolov5s.pt')
            print("Loaded YOLOv5s using ultralytics")
        except:
            print("Failed to load any YOLO model, using dummy detector")
            return None

    # Set device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if model is not None:
        try:
            model.to(device)
        except:
            pass
        print(f"YOLOv5 loaded on {device}")
    return model

# Initialize YOLOv5
print("Initializing YOLOv5...")
yolo_model = install_yolov5()

# Configure YOLOv5 for better performance
if yolo_model is not None:
    try:
        yolo_model.conf = 0.25  # Lower confidence threshold for better detection
        yolo_model.iou = 0.45   # IoU threshold for NMS
        # Include more vehicle classes
        yolo_model.classes = [0, 1, 2, 3, 4, 5, 6, 7, 8]  # person, bicycle, car, motorcycle, airplane, bus, train, truck, boat
    except:
        print("Using default YOLO settings")

# Step 1: Camera calibration and undistortion
def undistort_img():
    """Camera calibration function"""
    objp = np.zeros((6*9, 3), dtype=np.float32)
    objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

    objpoints = []
    imgpoints = []

    # Create camera_cal directory if it doesn't exist
    os.makedirs('camera_cal', exist_ok=True)

    images = glob.glob('camera_cal/*.jpg')

    if len(images) == 0:
        print("No calibration images found. Creating dummy calibration...")
        # Create dummy calibration parameters for demo
        img_size = (1280, 720)
        mtx = np.array([[1.15777930e+03, 0.00000000e+00, 6.67111054e+02],
                       [0.00000000e+00, 1.15282291e+03, 3.86128938e+02],
                       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
        dist = np.array([[-2.42565104e-01, -4.77893070e-02, -1.31388084e-03, -8.79107779e-05, 2.20573263e-02]])

        dist_pickle = {'mtx': mtx, 'dist': dist}
        pickle.dump(dist_pickle, open('camera_cal/cal_pickle.p', 'wb'))
        print("Dummy calibration created successfully")
        return

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)

        if ret:
            objpoints.append(objp.copy())
            imgpoints.append(corners)

    if len(objpoints) > 0:
        img_size = (img.shape[1], img.shape[0])
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

        dist_pickle = {'mtx': mtx, 'dist': dist}
        pickle.dump(dist_pickle, open('camera_cal/cal_pickle.p', 'wb'))
        print("Camera calibration successful:", ret)
    else:
        print("No valid calibration images found")

def undistort(img, cal_dir='camera_cal/cal_pickle.p'):
    """Undistort image using calibration parameters"""
    try:
        with open(cal_dir, 'rb') as f:
            calib = pickle.load(f)
        mtx = calib['mtx']
        dist = calib['dist']
        return cv2.undistort(img, mtx, dist, None, mtx)
    except:
        return img  # Return original if calibration fails

# Step 2: Enhanced image processing pipeline
def pipeline(img, s_thresh=(100, 255), sx_thresh=(15, 255)):
    """Enhanced image processing pipeline with better filtering"""
    img = undistort(img)
    img = np.copy(img)

    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(float)
    h_channel = hls[:, :, 0]
    l_channel = hls[:, :, 1]
    s_channel = hls[:, :, 2]

    # Enhanced yellow line detection
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    # Yellow mask
    yellow_lower = np.array([15, 100, 120])
    yellow_upper = np.array([35, 255, 255])
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)

    # White mask
    white_lower = np.array([0, 0, 200])
    white_upper = np.array([255, 30, 255])
    white_mask = cv2.inRange(hsv, white_lower, white_upper)

    # Sobel x gradient
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=7)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255 * abs_sobelx / np.max(abs_sobelx))

    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # S channel threshold
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Combine all thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1) | (yellow_mask == 255) | (white_mask == 255)] = 1

    return combined_binary

# Step 3: Perspective transformation
def perspective_warp(img,
                    dst_size=(1280, 720),
                    src=np.float32([(0.43, 0.65), (0.58, 0.65), (0.1, 1), (1, 1)]),
                    dst=np.float32([(0, 0), (1, 0), (0, 1), (1, 1)])):
    img_size = np.float32([(img.shape[1], img.shape[0])])
    src = src * img_size
    dst = dst * np.float32(dst_size)
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, dst_size)
    return warped

def inv_perspective_warp(img,
                         dst_size=(1280, 720),
                         src=np.float32([(0, 0), (1, 0), (0, 1), (1, 1)]),
                         dst=np.float32([(0.43, 0.65), (0.58, 0.65), (0.1, 1), (1, 1)])):
    img_size = np.float32([(img.shape[1], img.shape[0])])
    src = src * img_size
    dst = dst * np.float32(dst_size)
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, dst_size)
    return warped

# Global variables for lane tracking
left_a, left_b, left_c = [], [], []
right_a, right_b, right_c = [], [], []

# Step 4: Enhanced Vehicle detection with multiple approaches
def detect_vehicles(img):
    """Detect vehicles using YOLOv5 with multiple fallback approaches"""
    vehicles = []

    if yolo_model is None:
        return vehicles

    try:
        # Method 1: Try ultralytics YOLO approach
        try:
            from ultralytics import YOLO
            if hasattr(yolo_model, 'predict'):
                results = yolo_model.predict(img, conf=0.25, verbose=False)
                for result in results:
                    boxes = result.boxes
                    if boxes is not None:
                        for box in boxes:
                            # Get box coordinates
                            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                            conf = box.conf[0].cpu().numpy()
                            cls = int(box.cls[0].cpu().numpy())

                            # COCO class names
                            class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane',
                                         'bus', 'train', 'truck', 'boat', 'traffic light']

                            if cls < len(class_names):
                                class_name = class_names[cls]
                                # Only keep vehicle-related detections
                                if class_name in ['car', 'truck', 'bus', 'motorcycle', 'bicycle']:
                                    vehicles.append({
                                        'bbox': [int(x1), int(y1), int(x2), int(y2)],
                                        'confidence': float(conf),
                                        'class': class_name
                                    })
                return vehicles
        except Exception as e:
            print(f"Ultralytics approach failed: {e}")

        # Method 2: Try traditional torch hub approach
        try:
            # Convert BGR to RGB for YOLOv5
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Run inference
            results = yolo_model(rgb_img, size=640)

            # Parse results - try pandas format first
            try:
                detections = results.pandas().xyxy[0]
                for _, detection in detections.iterrows():
                    class_name = detection['name']
                    if class_name in ['car', 'truck', 'bus', 'motorcycle', 'bicycle', 'person']:
                        vehicles.append({
                            'bbox': [int(detection['xmin']), int(detection['ymin']),
                                    int(detection['xmax']), int(detection['ymax'])],
                            'confidence': float(detection['confidence']),
                            'class': class_name
                        })
            except:
                # Try tensor format
                detections = results.xyxy[0].cpu().numpy()
                class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane',
                             'bus', 'train', 'truck', 'boat']

                for detection in detections:
                    x1, y1, x2, y2, conf, cls = detection
                    cls = int(cls)
                    if cls < len(class_names):
                        class_name = class_names[cls]
                        if class_name in ['car', 'truck', 'bus', 'motorcycle', 'bicycle', 'person']:
                            vehicles.append({
                                'bbox': [int(x1), int(y1), int(x2), int(y2)],
                                'confidence': float(conf),
                                'class': class_name
                            })

            return vehicles

        except Exception as e:
            print(f"Traditional approach failed: {e}")
            return vehicles

    except Exception as e:
        print(f"Vehicle detection error: {e}")
        return vehicles

def draw_vehicle_detections(img, vehicles):
    """Draw vehicle detection boxes with distance calculation and collision warnings"""
    colors = {
        'car': (0, 255, 255),      # Yellow
        'truck': (255, 0, 255),    # Magenta
        'bus': (255, 165, 0),      # Orange
        'motorcycle': (0, 255, 0), # Green
        'bicycle': (255, 0, 0),    # Blue
        'person': (0, 0, 255)      # Red
    }

    img_height, img_width = img.shape[:2]
    vehicles_in_front = []

    for vehicle in vehicles:
        bbox = vehicle['bbox']
        confidence = vehicle['confidence']
        class_name = vehicle['class']

        # Calculate distance
        distance = calculate_vehicle_distance(bbox, img_height)

        # Check if vehicle is in front
        in_front = is_vehicle_in_front(bbox, img_width)

        if in_front and class_name in ['car', 'truck', 'bus']:
            vehicles_in_front.append({'vehicle': vehicle, 'distance': distance})

        # Get color for this class
        color = colors.get(class_name, (0, 255, 255))

        # Determine box thickness based on distance and position
        thickness = 5 if (in_front and distance < 20) else 3

        # Draw bounding box
        cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, thickness)

        # Prepare label with distance
        label = f"{class_name}: {confidence:.2f}"
        distance_label = f"Dist: {distance:.1f}m"

        # Draw labels
        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        dist_label_size = cv2.getTextSize(distance_label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]

        # Main label background
        cv2.rectangle(img, (bbox[0], bbox[1] - label_size[1] - 25),
                     (bbox[0] + max(label_size[0], dist_label_size[0]) + 10, bbox[1]), color, -1)

        # Draw label text
        cv2.putText(img, label, (bbox[0] + 5, bbox[1] - 15),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        cv2.putText(img, distance_label, (bbox[0] + 5, bbox[1] - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

        # Draw collision warning for vehicles too close
        if in_front and distance < 15 and class_name in ['car', 'truck', 'bus']:
            # Draw warning above the vehicle
            warning_y = max(bbox[1] - 60, 30)

            if distance < 8:  # Critical distance
                warning_color = (0, 0, 255)  # Red
                warning_text = "TOO CLOSE!"
            elif distance < 12:  # Warning distance
                warning_color = (0, 165, 255)  # Orange
                warning_text = "CLOSE!"
            else:  # Caution distance
                warning_color = (0, 255, 255)  # Yellow
                warning_text = "CAUTION"

            # Warning box
            warning_size = cv2.getTextSize(warning_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
            cv2.rectangle(img, (bbox[0], warning_y - 25),
                         (bbox[0] + warning_size[0] + 10, warning_y + 5), warning_color, -1)
            cv2.rectangle(img, (bbox[0], warning_y - 25),
                         (bbox[0] + warning_size[0] + 10, warning_y + 5), (255, 255, 255), 2)

            # Warning text
            cv2.putText(img, warning_text, (bbox[0] + 5, warning_y - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Draw overall collision warning if any vehicle is too close
    if vehicles_in_front:
        closest_distance = min([v['distance'] for v in vehicles_in_front])
        if closest_distance < 10:
            # Draw main collision warning
            cv2.rectangle(img, (img_width - 300, 50), (img_width - 50, 120), (0, 0, 255), -1)
            cv2.rectangle(img, (img_width - 300, 50), (img_width - 50, 120), (255, 255, 255), 3)

            cv2.putText(img, "COLLISION", (img_width - 280, 80),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(img, "WARNING!", (img_width - 270, 105),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    return img

# Step 5: Enhanced sliding window with better lane detection
def get_hist(img):
    return np.sum(img[img.shape[0]//2:, :], axis=0)

def sliding_window(img, nwindows=9, margin=120, minpix=50, draw_windows=False):
    global left_a, left_b, left_c, right_a, right_b, right_c

    left_fit_ = np.empty(3)
    right_fit_ = np.empty(3)
    out_img = np.dstack((img, img, img)) * 255

    histogram = get_hist(img)
    midpoint = int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    window_height = int(img.shape[0] / nwindows)

    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        win_y_low = img.shape[0] - (window + 1) * window_height
        win_y_high = img.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        if draw_windows:
            cv2.rectangle(out_img, (win_xleft_low, win_y_low),
                         (win_xleft_high, win_y_high), (100, 255, 255), 3)
            cv2.rectangle(out_img, (win_xright_low, win_y_low),
                         (win_xright_high, win_y_high), (100, 255, 255), 3)

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    if len(leftx) > 0 and len(rightx) > 0:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        left_a.append(left_fit[0])
        left_b.append(left_fit[1])
        left_c.append(left_fit[2])

        right_a.append(right_fit[0])
        right_b.append(right_fit[1])
        right_c.append(right_fit[2])

        left_fit_[0] = np.mean(left_a[-10:])
        left_fit_[1] = np.mean(left_b[-10:])
        left_fit_[2] = np.mean(left_c[-10:])

        right_fit_[0] = np.mean(right_a[-10:])
        right_fit_[1] = np.mean(right_b[-10:])
        right_fit_[2] = np.mean(right_c[-10:])
    else:
        left_fit_ = [0, 0, leftx_base] if len(left_a) == 0 else [np.mean(left_a[-5:]), np.mean(left_b[-5:]), np.mean(left_c[-5:])]
        right_fit_ = [0, 0, rightx_base] if len(right_a) == 0 else [np.mean(right_a[-5:]), np.mean(right_b[-5:]), np.mean(right_c[-5:])]

    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])
    left_fitx = left_fit_[0] * ploty ** 2 + left_fit_[1] * ploty + left_fit_[2]
    right_fitx = right_fit_[0] * ploty ** 2 + right_fit_[1] * ploty + right_fit_[2]

    if len(leftx) > 0:
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 100]
    if len(rightx) > 0:
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 100, 255]

    return out_img, (left_fitx, right_fitx), (left_fit_, right_fit_), ploty

# Step 6: Enhanced curvature calculation with curve warning
def get_curve(img, leftx, rightx):
    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])
    y_eval = np.max(ploty)
    ym_per_pix = 30.5 / 720
    xm_per_pix = 3.7 / 720

    left_fit_cr = np.polyfit(ploty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, rightx * xm_per_pix, 2)

    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * left_fit_cr[0] + 1e-6)
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * right_fit_cr[0] + 1e-6)

    car_pos = img.shape[1] / 2
    l_fit_x_int = left_fit_cr[0] * img.shape[0] ** 2 + left_fit_cr[1] * img.shape[0] + left_fit_cr[2]
    r_fit_x_int = right_fit_cr[0] * img.shape[0] ** 2 + right_fit_cr[1] * img.shape[0] + right_fit_cr[2]
    lane_center_position = (r_fit_x_int + l_fit_x_int) / 2
    center = (car_pos - lane_center_position) * xm_per_pix / 10

    return left_curverad, right_curverad, center

# Global variables for curve detection
curve_warning_counter = 0
last_curvature_values = []

def analyze_curve_severity(curvature):
    """Analyze curve severity and return warning level"""
    global last_curvature_values

    # Store last 10 curvature values for smoothing
    last_curvature_values.append(curvature)
    if len(last_curvature_values) > 10:
        last_curvature_values.pop(0)

    avg_curvature = np.mean(last_curvature_values)

    if avg_curvature < 200:
        return "SHARP_CURVE"
    elif avg_curvature < 400:
        return "MODERATE_CURVE"
    elif avg_curvature < 800:
        return "GENTLE_CURVE"
    else:
        return "STRAIGHT"

def draw_lanes(img, left_fit, right_fit):
    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])
    color_img = np.zeros_like(img)

    left_points = np.array([np.transpose(np.vstack([left_fit, ploty]))])
    right_points = np.array([np.flipud(np.transpose(np.vstack([right_fit, ploty])))])
    points = np.hstack((left_points, right_points))

    cv2.fillPoly(color_img, np.int_(points), (0, 255, 0))
    inv_perspective = inv_perspective_warp(color_img)
    img_out = cv2.addWeighted(img, 1, inv_perspective, 0.7, 0)
    return img_out

def calculate_vehicle_distance(bbox, img_height):
    """Calculate approximate distance to vehicle based on bounding box position and size"""
    # Vehicle bottom y-coordinate
    vehicle_bottom_y = bbox[3]

    # Vehicle height in pixels
    vehicle_height = bbox[3] - bbox[1]

    # Approximate distance calculation (empirical formula)
    # Assumes average car height of 1.5 meters
    if vehicle_height > 0:
        # Distance in meters (approximate)
        distance = (1.5 * 720) / vehicle_height  # Normalized for 720p

        # Adjust based on vertical position (perspective effect)
        y_factor = (img_height - vehicle_bottom_y) / img_height
        distance = distance * (0.5 + y_factor * 1.5)

        return max(distance, 1.0)  # Minimum 1 meter

    return 50.0  # Default distance if calculation fails

def is_vehicle_in_front(bbox, img_width):
    """Check if vehicle is directly in front (in ego lane)"""
    vehicle_center_x = (bbox[0] + bbox[2]) / 2
    img_center_x = img_width / 2

    # Check if vehicle is in the center lane area (±25% of image width)
    lane_width_tolerance = img_width * 0.25

    return abs(vehicle_center_x - img_center_x) < lane_width_tolerance

def draw_enhanced_curve_warning(img, curvature):
    """Draw enhanced curve warning with different severity levels"""
    global curve_warning_counter

    curve_type = analyze_curve_severity(curvature)

    if curve_type != "STRAIGHT":
        curve_warning_counter = max(curve_warning_counter, 30)  # Show for at least 30 frames

    if curve_warning_counter > 0:
        curve_warning_counter -= 1

        # Color and message based on severity
        if curve_type == "SHARP_CURVE":
            color = (0, 0, 255)  # Red
            warning_msg = "SHARP CURVE AHEAD!"
            sub_msg = "REDUCE SPEED"
        elif curve_type == "MODERATE_CURVE":
            color = (0, 165, 255)  # Orange
            warning_msg = "CURVE AHEAD!"
            sub_msg = "SLOW DOWN"
        else:  # GENTLE_CURVE
            color = (0, 255, 255)  # Yellow
            warning_msg = "GENTLE CURVE"
            sub_msg = "CAUTION"

        # Draw warning box with pulsing effect
        pulse = int(20 * (1 + 0.3 * np.sin(curve_warning_counter * 0.3)))

        cv2.rectangle(img, (50, 50), (450, 130), color, -1)
        cv2.rectangle(img, (50, 50), (450, 130), (255, 255, 255), 4)

        # Warning text
        cv2.putText(img, warning_msg, (70, 85),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 3)
        cv2.putText(img, sub_msg, (150, 115),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Curvature value
        cv2.putText(img, f"R: {curvature:.0f}m", (350, 115),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return img

# Step 7: Enhanced video processing pipeline
def vid_pipeline(img):
    """Main video processing pipeline with enhanced curve detection and collision warnings"""
    try:
        # Lane detection
        img_bin = pipeline(img)
        img_warp = perspective_warp(img_bin)
        out_img, curves, lanes, ploty = sliding_window(img_warp, draw_windows=False)

        # Calculate curvature
        curverad = get_curve(img, curves[0], curves[1])
        lane_curve = np.mean([curverad[0], curverad[1]])

        # Draw lanes
        img_out = draw_lanes(img, curves[0], curves[1])

        # Vehicle detection and distance calculation
        vehicles = detect_vehicles(img_out)
        img_out = draw_vehicle_detections(img_out, vehicles)

        # Enhanced curve warning system
        img_out = draw_enhanced_curve_warning(img_out, lane_curve)

        # Enhanced information display with better background
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontColor = (255, 255, 255)
        fontSize = 0.6
        thickness = 2

        # Larger background for text with transparency effect
        overlay = img_out.copy()
        cv2.rectangle(overlay, (10, img_out.shape[0] - 120), (500, img_out.shape[0] - 10), (0, 0, 0), -1)
        img_out = cv2.addWeighted(img_out, 0.7, overlay, 0.3, 0)

        # Display enhanced information
        cv2.putText(img_out, f'Lane Curvature: {lane_curve:.0f} m',
                   (20, img_out.shape[0] - 95), font, fontSize, fontColor, thickness)
        cv2.putText(img_out, f'Vehicle Offset: {curverad[2]:.3f} m',
                   (20, img_out.shape[0] - 70), font, fontSize, fontColor, thickness)
        cv2.putText(img_out, f'Vehicles Detected: {len(vehicles)}',
                   (20, img_out.shape[0] - 45), font, fontSize, fontColor, thickness)

        # Add curve type information
        curve_type = analyze_curve_severity(lane_curve)
        curve_status = curve_type.replace('_', ' ').title()
        cv2.putText(img_out, f'Road Status: {curve_status}',
                   (20, img_out.shape[0] - 20), font, fontSize, fontColor, thickness)

        return img_out

    except Exception as e:
        print(f"Pipeline error: {e}")
        return img

# Step 8: Main execution with better error handling
def main():
    """Main function to run the enhanced lane detection"""
    # Run calibration
    print("Running camera calibration...")
    undistort_img()

    # Test YOLO model with a sample
    print("Testing YOLO model...")
    if yolo_model is not None:
        try:
            # Create a test image
            test_img = np.ones((480, 640, 3), dtype=np.uint8) * 128
            test_vehicles = detect_vehicles(test_img)
            print(f"YOLO model test: {len(test_vehicles)} vehicles detected in test image")
        except Exception as e:
            print(f"YOLO test failed: {e}")
    else:
        print("Warning: YOLO model not loaded properly")

    # Check if video exists
    video_path = 'project_video.mp4'
    if not os.path.exists(video_path):
        # Try alternative names
        alternative_paths = ['project_video.mov', 'video.mp4', 'test_video.mp4', 'input.mp4']
        found = False
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                video_path = alt_path
                found = True
                print(f"Found video: {video_path}")
                break

        if not found:
            print(f"Video file not found. Tried: {video_path}, {', '.join(alternative_paths)}")
            print("Please upload your video file and update the video_path variable.")
            return

    # Process video
    print("Processing video...")
    try:
        myclip = VideoFileClip(video_path)
        output_vid = 'enhanced_lane_detection_output.mp4'

        print(f"Video duration: {myclip.duration:.2f} seconds")
        print(f"Video size: {myclip.size}")
        print(f"Video FPS: {myclip.fps}")

        # Process with higher quality settings
        clip = myclip.fl_image(vid_pipeline)
        clip.write_videofile(output_vid,
                           audio=False,
                           codec='libx264',
                           temp_audiofile='temp-audio.m4a',
                           remove_temp=True,
                           verbose=False,
                           logger=None)

        print(f"Video processing complete! Output saved as: {output_vid}")

        # Display video
        return HTML(f"""
        <video width="1280" height="720" controls>
          <source src="{output_vid}" type="video/mp4">
        </video>
        """)

    except Exception as e:
        print(f"Video processing error: {e}")
        print("Make sure your video file exists and is accessible.")
        import traceback
        traceback.print_exc()

# Run the main function
if __name__ == "__main__":
    result = main()
    if result:
        display(result)

Initializing YOLOv5...
Ultralytics installed successfully
Custom model not found, using YOLOv5s pretrained...


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-7-26 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 216MB/s]

Fusing layers... 


In [ ]:

%cd /content/lane_detection
%cd Curved-Lane-Lines

/content/lane_detection
/content/lane_detection/Curved-Lane-Lines


In [ ]:
# Import necessary libraries
import numpy as np
import cv2
import glob
import pickle
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import torch
import torchvision
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Install YOLOv5 if not already installed
import subprocess
import sys
import os

def install_yolov5():
    """Install YOLOv5 dependencies"""
    try:
        # Try to install ultralytics first (more reliable)
        subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics", "-q"])
        print("Ultralytics installed successfully")
    except:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "yolov5", "-q"])
            print("YOLOv5 installed successfully")
        except:
            print("Failed to install YOLOv5 packages")

    try:
        # Try custom model first
        if os.path.exists('best.pt'):
            print("Loading custom YOLOv5 model...")
            model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True, trust_repo=True)
        else:
            print("Custom model not found, using YOLOv5s pretrained...")
            model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, trust_repo=True)
    except Exception as e:
        print(f"Error loading model from hub: {e}")
        print("Trying alternative approach...")
        try:
            # Alternative: use ultralytics YOLO
            from ultralytics import YOLO
            model = YOLO('yolov5s.pt')
            print("Loaded YOLOv5s using ultralytics")
        except:
            print("Failed to load any YOLO model, using dummy detector")
            return None

    # Set device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if model is not None:
        try:
            model.to(device)
        except:
            pass
        print(f"YOLOv5 loaded on {device}")
    return model

# Initialize YOLOv5
print("Initializing YOLOv5...")
yolo_model = install_yolov5()

# Configure YOLOv5 for better performance
if yolo_model is not None:
    try:
        yolo_model.conf = 0.25  # Lower confidence threshold for better detection
        yolo_model.iou = 0.45   # IoU threshold for NMS
        # Include more vehicle classes
        yolo_model.classes = [0, 1, 2, 3, 4, 5, 6, 7, 8]  # person, bicycle, car, motorcycle, airplane, bus, train, truck, boat
    except:
        print("Using default YOLO settings")

# Step 1: Camera calibration and undistortion
def undistort_img():
    """Camera calibration function"""
    objp = np.zeros((6*9, 3), dtype=np.float32)
    objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

    objpoints = []
    imgpoints = []

    # Create camera_cal directory if it doesn't exist
    os.makedirs('camera_cal', exist_ok=True)

    images = glob.glob('camera_cal/*.jpg')

    if len(images) == 0:
        print("No calibration images found. Creating dummy calibration...")
        # Create dummy calibration parameters for demo
        img_size = (1280, 720)
        mtx = np.array([[1.15777930e+03, 0.00000000e+00, 6.67111054e+02],
                       [0.00000000e+00, 1.15282291e+03, 3.86128938e+02],
                       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
        dist = np.array([[-2.42565104e-01, -4.77893070e-02, -1.31388084e-03, -8.79107779e-05, 2.20573263e-02]])

        dist_pickle = {'mtx': mtx, 'dist': dist}
        pickle.dump(dist_pickle, open('camera_cal/cal_pickle.p', 'wb'))
        print("Dummy calibration created successfully")
        return

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)

        if ret:
            objpoints.append(objp.copy())
            imgpoints.append(corners)

    if len(objpoints) > 0:
        img_size = (img.shape[1], img.shape[0])
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

        dist_pickle = {'mtx': mtx, 'dist': dist}
        pickle.dump(dist_pickle, open('camera_cal/cal_pickle.p', 'wb'))
        print("Camera calibration successful:", ret)
    else:
        print("No valid calibration images found")

def undistort(img, cal_dir='camera_cal/cal_pickle.p'):
    """Undistort image using calibration parameters"""
    try:
        with open(cal_dir, 'rb') as f:
            calib = pickle.load(f)
        mtx = calib['mtx']
        dist = calib['dist']
        return cv2.undistort(img, mtx, dist, None, mtx)
    except:
        return img  # Return original if calibration fails

# Step 2: Enhanced image processing pipeline
def pipeline(img, s_thresh=(100, 255), sx_thresh=(15, 255)):
    """Enhanced image processing pipeline with better filtering"""
    img = undistort(img)
    img = np.copy(img)

    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(float)
    h_channel = hls[:, :, 0]
    l_channel = hls[:, :, 1]
    s_channel = hls[:, :, 2]

    # Enhanced yellow line detection
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    # Yellow mask
    yellow_lower = np.array([15, 100, 120])
    yellow_upper = np.array([35, 255, 255])
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)

    # White mask
    white_lower = np.array([0, 0, 200])
    white_upper = np.array([255, 30, 255])
    white_mask = cv2.inRange(hsv, white_lower, white_upper)

    # Sobel x gradient
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=7)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255 * abs_sobelx / np.max(abs_sobelx))

    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # S channel threshold
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Combine all thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1) | (yellow_mask == 255) | (white_mask == 255)] = 1

    return combined_binary

# Step 3: Perspective transformation
def perspective_warp(img,
                    dst_size=(1280, 720),
                    src=np.float32([(0.43, 0.65), (0.58, 0.65), (0.1, 1), (1, 1)]),
                    dst=np.float32([(0, 0), (1, 0), (0, 1), (1, 1)])):
    img_size = np.float32([(img.shape[1], img.shape[0])])
    src = src * img_size
    dst = dst * np.float32(dst_size)
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, dst_size)
    return warped

def inv_perspective_warp(img,
                         dst_size=(1280, 720),
                         src=np.float32([(0, 0), (1, 0), (0, 1), (1, 1)]),
                         dst=np.float32([(0.43, 0.65), (0.58, 0.65), (0.1, 1), (1, 1)])):
    img_size = np.float32([(img.shape[1], img.shape[0])])
    src = src * img_size
    dst = dst * np.float32(dst_size)
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, dst_size)
    return warped

# Global variables for lane tracking
left_a, left_b, left_c = [], [], []
right_a, right_b, right_c = [], [], []

# Step 4: Enhanced Vehicle detection with multiple approaches
def detect_vehicles(img):
    """Detect vehicles using YOLOv5 with multiple fallback approaches"""
    vehicles = []

    if yolo_model is None:
        return vehicles

    try:
        # Method 1: Try ultralytics YOLO approach
        try:
            from ultralytics import YOLO
            if hasattr(yolo_model, 'predict'):
                results = yolo_model.predict(img, conf=0.25, verbose=False)
                for result in results:
                    boxes = result.boxes
                    if boxes is not None:
                        for box in boxes:
                            # Get box coordinates
                            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                            conf = box.conf[0].cpu().numpy()
                            cls = int(box.cls[0].cpu().numpy())

                            # COCO class names
                            class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane',
                                         'bus', 'train', 'truck', 'boat', 'traffic light']

                            if cls < len(class_names):
                                class_name = class_names[cls]
                                # Only keep vehicle-related detections
                                if class_name in ['car', 'truck', 'bus', 'motorcycle', 'bicycle']:
                                    vehicles.append({
                                        'bbox': [int(x1), int(y1), int(x2), int(y2)],
                                        'confidence': float(conf),
                                        'class': class_name
                                    })
                return vehicles
        except Exception as e:
            print(f"Ultralytics approach failed: {e}")

        # Method 2: Try traditional torch hub approach
        try:
            # Convert BGR to RGB for YOLOv5
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Run inference
            results = yolo_model(rgb_img, size=640)

            # Parse results - try pandas format first
            try:
                detections = results.pandas().xyxy[0]
                for _, detection in detections.iterrows():
                    class_name = detection['name']
                    if class_name in ['car', 'truck', 'bus', 'motorcycle', 'bicycle', 'person']:
                        vehicles.append({
                            'bbox': [int(detection['xmin']), int(detection['ymin']),
                                    int(detection['xmax']), int(detection['ymax'])],
                            'confidence': float(detection['confidence']),
                            'class': class_name
                        })
            except:
                # Try tensor format
                detections = results.xyxy[0].cpu().numpy()
                class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane',
                             'bus', 'train', 'truck', 'boat']

                for detection in detections:
                    x1, y1, x2, y2, conf, cls = detection
                    cls = int(cls)
                    if cls < len(class_names):
                        class_name = class_names[cls]
                        if class_name in ['car', 'truck', 'bus', 'motorcycle', 'bicycle', 'person']:
                            vehicles.append({
                                'bbox': [int(x1), int(y1), int(x2), int(y2)],
                                'confidence': float(conf),
                                'class': class_name
                            })

            return vehicles

        except Exception as e:
            print(f"Traditional approach failed: {e}")
            return vehicles

    except Exception as e:
        print(f"Vehicle detection error: {e}")
        return vehicles

def draw_vehicle_detections(img, vehicles):
    """Draw vehicle detection boxes with distance calculation and collision warnings"""
    colors = {
        'car': (0, 255, 255),      # Yellow
        'truck': (255, 0, 255),    # Magenta
        'bus': (255, 165, 0),      # Orange
        'motorcycle': (0, 255, 0), # Green
        'bicycle': (255, 0, 0),    # Blue
        'person': (0, 0, 255)      # Red
    }

    img_height, img_width = img.shape[:2]
    vehicles_in_front = []

    for vehicle in vehicles:
        bbox = vehicle['bbox']
        confidence = vehicle['confidence']
        class_name = vehicle['class']

        # Calculate distance
        distance = calculate_vehicle_distance(bbox, img_height)

        # Check if vehicle is in front
        in_front = is_vehicle_in_front(bbox, img_width)

        if in_front and class_name in ['car', 'truck', 'bus']:
            vehicles_in_front.append({'vehicle': vehicle, 'distance': distance})

        # Get color for this class
        color = colors.get(class_name, (0, 255, 255))

        # Determine box thickness based on distance and position
        thickness = 5 if (in_front and distance < 20) else 3

        # Draw bounding box
        cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, thickness)

        # Prepare label with distance
        label = f"{class_name}: {confidence:.2f}"
        distance_label = f"Dist: {distance:.1f}m"

        # Draw labels
        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        dist_label_size = cv2.getTextSize(distance_label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]

        # Main label background
        cv2.rectangle(img, (bbox[0], bbox[1] - label_size[1] - 25),
                     (bbox[0] + max(label_size[0], dist_label_size[0]) + 10, bbox[1]), color, -1)

        # Draw label text
        cv2.putText(img, label, (bbox[0] + 5, bbox[1] - 15),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        cv2.putText(img, distance_label, (bbox[0] + 5, bbox[1] - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

        # Draw collision warning for vehicles too close
        if in_front and distance < 15 and class_name in ['car', 'truck', 'bus']:
            # Draw warning above the vehicle
            warning_y = max(bbox[1] - 60, 30)

            if distance < 8:  # Critical distance
                warning_color = (0, 0, 255)  # Red
                warning_text = "TOO CLOSE!"
            elif distance < 12:  # Warning distance
                warning_color = (0, 165, 255)  # Orange
                warning_text = "CLOSE!"
            else:  # Caution distance
                warning_color = (0, 255, 255)  # Yellow
                warning_text = "CAUTION"

            # Warning box
            warning_size = cv2.getTextSize(warning_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
            cv2.rectangle(img, (bbox[0], warning_y - 25),
                         (bbox[0] + warning_size[0] + 10, warning_y + 5), warning_color, -1)
            cv2.rectangle(img, (bbox[0], warning_y - 25),
                         (bbox[0] + warning_size[0] + 10, warning_y + 5), (255, 255, 255), 2)

            # Warning text
            cv2.putText(img, warning_text, (bbox[0] + 5, warning_y - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Draw overall collision warning if any vehicle is too close
    if vehicles_in_front:
        closest_distance = min([v['distance'] for v in vehicles_in_front])
        if closest_distance < 10:
            # Draw main collision warning
            cv2.rectangle(img, (img_width - 300, 50), (img_width - 50, 120), (0, 0, 255), -1)
            cv2.rectangle(img, (img_width - 300, 50), (img_width - 50, 120), (255, 255, 255), 3)

            cv2.putText(img, "COLLISION", (img_width - 280, 80),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(img, "WARNING!", (img_width - 270, 105),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    return img

# Step 5: Enhanced sliding window with better lane detection
def get_hist(img):
    return np.sum(img[img.shape[0]//2:, :], axis=0)

def sliding_window(img, nwindows=9, margin=120, minpix=50, draw_windows=False):
    global left_a, left_b, left_c, right_a, right_b, right_c

    left_fit_ = np.empty(3)
    right_fit_ = np.empty(3)
    out_img = np.dstack((img, img, img)) * 255

    histogram = get_hist(img)
    midpoint = int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    window_height = int(img.shape[0] / nwindows)

    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        win_y_low = img.shape[0] - (window + 1) * window_height
        win_y_high = img.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        if draw_windows:
            cv2.rectangle(out_img, (win_xleft_low, win_y_low),
                         (win_xleft_high, win_y_high), (100, 255, 255), 3)
            cv2.rectangle(out_img, (win_xright_low, win_y_low),
                         (win_xright_high, win_y_high), (100, 255, 255), 3)

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    if len(leftx) > 0 and len(rightx) > 0:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        left_a.append(left_fit[0])
        left_b.append(left_fit[1])
        left_c.append(left_fit[2])

        right_a.append(right_fit[0])
        right_b.append(right_fit[1])
        right_c.append(right_fit[2])

        left_fit_[0] = np.mean(left_a[-10:])
        left_fit_[1] = np.mean(left_b[-10:])
        left_fit_[2] = np.mean(left_c[-10:])

        right_fit_[0] = np.mean(right_a[-10:])
        right_fit_[1] = np.mean(right_b[-10:])
        right_fit_[2] = np.mean(right_c[-10:])
    else:
        left_fit_ = [0, 0, leftx_base] if len(left_a) == 0 else [np.mean(left_a[-5:]), np.mean(left_b[-5:]), np.mean(left_c[-5:])]
        right_fit_ = [0, 0, rightx_base] if len(right_a) == 0 else [np.mean(right_a[-5:]), np.mean(right_b[-5:]), np.mean(right_c[-5:])]

    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])
    left_fitx = left_fit_[0] * ploty ** 2 + left_fit_[1] * ploty + left_fit_[2]
    right_fitx = right_fit_[0] * ploty ** 2 + right_fit_[1] * ploty + right_fit_[2]

    if len(leftx) > 0:
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 100]
    if len(rightx) > 0:
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 100, 255]

    return out_img, (left_fitx, right_fitx), (left_fit_, right_fit_), ploty

# Step 6: Enhanced curvature calculation with curve warning
def get_curve(img, leftx, rightx):
    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])
    y_eval = np.max(ploty)
    ym_per_pix = 30.5 / 720
    xm_per_pix = 3.7 / 720

    left_fit_cr = np.polyfit(ploty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, rightx * xm_per_pix, 2)

    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * left_fit_cr[0] + 1e-6)
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * right_fit_cr[0] + 1e-6)

    car_pos = img.shape[1] / 2
    l_fit_x_int = left_fit_cr[0] * img.shape[0] ** 2 + left_fit_cr[1] * img.shape[0] + left_fit_cr[2]
    r_fit_x_int = right_fit_cr[0] * img.shape[0] ** 2 + right_fit_cr[1] * img.shape[0] + right_fit_cr[2]
    lane_center_position = (r_fit_x_int + l_fit_x_int) / 2
    center = (car_pos - lane_center_position) * xm_per_pix / 10

    return left_curverad, right_curverad, center

# Global variables for curve detection
curve_warning_counter = 0
last_curvature_values = []

def analyze_curve_severity(curvature):
    """Analyze curve severity and return warning level"""
    global last_curvature_values

    # Store last 10 curvature values for smoothing
    last_curvature_values.append(curvature)
    if len(last_curvature_values) > 10:
        last_curvature_values.pop(0)

    avg_curvature = np.mean(last_curvature_values)

    if avg_curvature < 200:
        return "SHARP_CURVE"
    elif avg_curvature < 400:
        return "MODERATE_CURVE"
    elif avg_curvature < 800:
        return "GENTLE_CURVE"
    else:
        return "STRAIGHT"

def draw_lanes(img, left_fit, right_fit):
    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])
    color_img = np.zeros_like(img)

    left_points = np.array([np.transpose(np.vstack([left_fit, ploty]))])
    right_points = np.array([np.flipud(np.transpose(np.vstack([right_fit, ploty])))])
    points = np.hstack((left_points, right_points))

    cv2.fillPoly(color_img, np.int_(points), (0, 255, 0))
    inv_perspective = inv_perspective_warp(color_img)
    img_out = cv2.addWeighted(img, 1, inv_perspective, 0.7, 0)
    return img_out

def calculate_vehicle_distance(bbox, img_height):
    """Calculate approximate distance to vehicle based on bounding box position and size"""
    # Vehicle bottom y-coordinate
    vehicle_bottom_y = bbox[3]

    # Vehicle height in pixels
    vehicle_height = bbox[3] - bbox[1]

    # Approximate distance calculation (empirical formula)
    # Assumes average car height of 1.5 meters
    if vehicle_height > 0:
        # Distance in meters (approximate)
        distance = (1.5 * 720) / vehicle_height  # Normalized for 720p

        # Adjust based on vertical position (perspective effect)
        y_factor = (img_height - vehicle_bottom_y) / img_height
        distance = distance * (0.5 + y_factor * 1.5)

        return max(distance, 1.0)  # Minimum 1 meter

    return 50.0  # Default distance if calculation fails

def is_vehicle_in_front(bbox, img_width):
    """Check if vehicle is directly in front (in ego lane)"""
    vehicle_center_x = (bbox[0] + bbox[2]) / 2
    img_center_x = img_width / 2

    # Check if vehicle is in the center lane area (±25% of image width)
    lane_width_tolerance = img_width * 0.25

    return abs(vehicle_center_x - img_center_x) < lane_width_tolerance

def draw_enhanced_curve_warning(img, curvature):
    """Draw enhanced curve warning with improved styling"""
    global curve_warning_counter

    curve_type = analyze_curve_severity(curvature)

    if curve_type != "STRAIGHT":
        curve_warning_counter = max(curve_warning_counter, 30)  # Show for at least 30 frames

    if curve_warning_counter > 0:
        curve_warning_counter -= 1

        # Get image dimensions for centering
        img_height, img_width = img.shape[:2]

        # Warning box dimensions
        box_width = 380
        box_height = 80
        box_x = (img_width - box_width) // 2
        box_y = 40

        # Color and message based on severity
        if curve_type == "SHARP_CURVE":
            color = (0, 0, 255)  # Red
            warning_msg = "SHARP CURVE AHEAD!"
            sub_msg = "REDUCE SPEED NOW"
        elif curve_type == "MODERATE_CURVE":
            color = (0, 100, 255)  # Orange-Red
            warning_msg = "CURVE AHEAD!"
            sub_msg = "SLOW DOWN"
        else:  # GENTLE_CURVE
            color = (0, 200, 255)  # Yellow-Orange
            warning_msg = "GENTLE CURVE"
            sub_msg = "CAUTION"

        # Draw main warning box with gradient effect
        cv2.rectangle(img, (box_x, box_y), (box_x + box_width, box_y + box_height), color, -1)
        cv2.rectangle(img, (box_x, box_y), (box_x + box_width, box_y + box_height), (255, 255, 255), 4)

        # Inner shadow effect
        cv2.rectangle(img, (box_x + 5, box_y + 5), (box_x + box_width - 5, box_y + box_height - 5),
                     (255, 255, 255, 50), 2)

        # Warning text with better positioning
        # Main warning text
        text_size = cv2.getTextSize(warning_msg, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 3)[0]
        text_x = box_x + (box_width - text_size[0]) // 2
        cv2.putText(img, warning_msg, (text_x, box_y + 35),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 3)

        # Sub message
        sub_size = cv2.getTextSize(sub_msg, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
        sub_x = box_x + (box_width - sub_size[0]) // 2
        cv2.putText(img, sub_msg, (sub_x, box_y + 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Curvature value in top right corner of warning
        cv2.putText(img, f"R: {curvature:.0f}m", (box_x + box_width - 90, box_y + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    return img

def draw_vehicle_detections(img, vehicles):
    """Draw vehicle detection boxes with improved styling and distance calculation"""
    colors = {
        'car': (0, 255, 255),      # Cyan
        'truck': (128, 0, 255),    # Purple
        'bus': (0, 165, 255),      # Orange
        'motorcycle': (0, 255, 0), # Green
        'bicycle': (255, 0, 0),    # Blue
        'person': (0, 0, 255)      # Red
    }

    img_height, img_width = img.shape[:2]
    vehicles_in_front = []

    for vehicle in vehicles:
        bbox = vehicle['bbox']
        confidence = vehicle['confidence']
        class_name = vehicle['class']

        # Calculate distance
        distance = calculate_vehicle_distance(bbox, img_height)

        # Check if vehicle is in front
        in_front = is_vehicle_in_front(bbox, img_width)

        if in_front and class_name in ['car', 'truck', 'bus']:
            vehicles_in_front.append({'vehicle': vehicle, 'distance': distance})

        # Get color for this class
        color = colors.get(class_name, (0, 255, 255))

        # Determine box thickness based on distance and position
        thickness = 4 if (in_front and distance < 20) else 2

        # Draw bounding box with rounded corners effect
        cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, thickness)
        cv2.rectangle(img, (bbox[0] - 2, bbox[1] - 2), (bbox[2] + 2, bbox[3] + 2), (255, 255, 255), 1)

        # Create styled label background
        label = f"{class_name.upper()}: {confidence:.2f}"
        distance_label = f"DIST: {distance:.1f}m"

        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        dist_label_size = cv2.getTextSize(distance_label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]

        # Label background with proper spacing
        max_width = max(label_size[0], dist_label_size[0]) + 20
        label_height = 50

        # Main label background with gradient effect
        cv2.rectangle(img, (bbox[0], bbox[1] - label_height),
                     (bbox[0] + max_width, bbox[1]), color, -1)
        cv2.rectangle(img, (bbox[0], bbox[1] - label_height),
                     (bbox[0] + max_width, bbox[1]), (255, 255, 255), 2)

        # Draw label text with better positioning
        cv2.putText(img, label, (bbox[0] + 10, bbox[1] - 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        cv2.putText(img, distance_label, (bbox[0] + 10, bbox[1] - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

        # Draw enhanced collision warning for vehicles too close
        if in_front and distance < 15 and class_name in ['car', 'truck', 'bus']:
            draw_collision_warning(img, bbox, distance)

    # Draw overall collision warning if any vehicle is too close
    if vehicles_in_front:
        closest_distance = min([v['distance'] for v in vehicles_in_front])
        if closest_distance < 10:
            draw_main_collision_warning(img, closest_distance)

    return img

def draw_collision_warning(img, bbox, distance):
    """Draw individual collision warning above vehicle"""
    warning_y = max(bbox[1] - 80, 30)

    if distance < 8:  # Critical distance
        warning_color = (0, 0, 255)  # Red
        warning_text = "TOO CLOSE!"
    elif distance < 12:  # Warning distance
        warning_color = (0, 100, 255)  # Orange
        warning_text = "CLOSE!"
    else:  # Caution distance
        warning_color = (0, 200, 255)  # Yellow
        warning_text = "CAUTION"

    # Warning box with improved styling
    warning_size = cv2.getTextSize(warning_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
    box_width = warning_size[0] + 20
    box_height = 35

    cv2.rectangle(img, (bbox[0], warning_y - box_height),
                 (bbox[0] + box_width, warning_y), warning_color, -1)
    cv2.rectangle(img, (bbox[0], warning_y - box_height),
                 (bbox[0] + box_width, warning_y), (255, 255, 255), 3)

    # Warning text
    text_x = bbox[0] + 10
    cv2.putText(img, warning_text, (text_x, warning_y - 10),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

def draw_main_collision_warning(img, distance):
    """Draw main collision warning panel"""
    img_height, img_width = img.shape[:2]

    # Warning panel dimensions
    panel_width = 280
    panel_height = 80
    panel_x = img_width - panel_width - 20
    panel_y = 120

    # Draw warning panel
    cv2.rectangle(img, (panel_x, panel_y), (panel_x + panel_width, panel_y + panel_height),
                 (0, 0, 255), -1)
    cv2.rectangle(img, (panel_x, panel_y), (panel_x + panel_width, panel_y + panel_height),
                 (255, 255, 255), 4)

    # Warning text
    cv2.putText(img, "COLLISION WARNING!", (panel_x + 20, panel_y + 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(img, f"Distance: {distance:.1f}m", (panel_x + 50, panel_y + 60),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

# Step 7: Enhanced video processing pipeline with improved visualization
def vid_pipeline(img):
    """Main video processing pipeline with enhanced visualization and text layout"""
    try:
        # Lane detection
        img_bin = pipeline(img)
        img_warp = perspective_warp(img_bin)
        out_img, curves, lanes, ploty = sliding_window(img_warp, draw_windows=False)

        # Calculate curvature
        curverad = get_curve(img, curves[0], curves[1])
        lane_curve = np.mean([curverad[0], curverad[1]])

        # Draw lanes
        img_out = draw_lanes(img, curves[0], curves[1])

        # Vehicle detection and distance calculation
        vehicles = detect_vehicles(img_out)
        img_out = draw_vehicle_detections(img_out, vehicles)

        # Enhanced curve warning system
        img_out = draw_enhanced_curve_warning(img_out, lane_curve)

        # Get image dimensions
        img_height, img_width = img_out.shape[:2]

        # Create structured information panels
        draw_info_panels(img_out, lane_curve, curverad[2], len(vehicles))

        return img_out

    except Exception as e:
        print(f"Pipeline error: {e}")
        return img

def draw_info_panels(img, lane_curve, vehicle_offset, vehicle_count):
    """Draw well-structured information panels"""
    img_height, img_width = img.shape[:2]

    # Panel 1: Main Lane Information (Bottom Left)
    panel1_x = 15
    panel1_y = img_height - 140
    panel1_width = 280
    panel1_height = 125

    # Create semi-transparent panel background
    overlay = img.copy()
    cv2.rectangle(overlay, (panel1_x, panel1_y), (panel1_x + panel1_width, panel1_y + panel1_height),
                 (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.7, img, 0.3, 0, img)

    # Panel border
    cv2.rectangle(img, (panel1_x, panel1_y), (panel1_x + panel1_width, panel1_y + panel1_height),
                 (255, 255, 255), 2)

    # Panel title
    cv2.rectangle(img, (panel1_x, panel1_y), (panel1_x + panel1_width, panel1_y + 25),
                 (50, 50, 200), -1)
    cv2.putText(img, "LANE INFORMATION", (panel1_x + 10, panel1_y + 18),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Information text with proper spacing
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.55
    font_thickness = 2
    line_height = 22
    text_color = (255, 255, 255)

    # Lane curvature
    cv2.putText(img, f"Curvature:", (panel1_x + 10, panel1_y + 45),
               font, font_scale, text_color, font_thickness)
    cv2.putText(img, f"{lane_curve:.0f} m", (panel1_x + 140, panel1_y + 45),
               font, font_scale, (0, 255, 255), font_thickness)

    # Vehicle offset
    cv2.putText(img, f"Vehicle Offset:", (panel1_x + 10, panel1_y + 67),
               font, font_scale, text_color, font_thickness)
    offset_color = (0, 255, 0) if abs(vehicle_offset) < 0.1 else (0, 255, 255)
    cv2.putText(img, f"{vehicle_offset:.3f} m", (panel1_x + 140, panel1_y + 67),
               font, font_scale, offset_color, font_thickness)

    # Vehicle count
    cv2.putText(img, f"Vehicles:", (panel1_x + 10, panel1_y + 89),
               font, font_scale, text_color, font_thickness)
    count_color = (0, 255, 0) if vehicle_count == 0 else (0, 255, 255)
    cv2.putText(img, f"{vehicle_count}", (panel1_x + 140, panel1_y + 89),
               font, font_scale, count_color, font_thickness)

    # Road status
    curve_type = analyze_curve_severity(lane_curve)
    road_status = curve_type.replace('_', ' ').title()
    cv2.putText(img, f"Road Status:", (panel1_x + 10, panel1_y + 111),
               font, font_scale, text_color, font_thickness)

    # Color-code road status
    if curve_type == "SHARP_CURVE":
        status_color = (0, 0, 255)
    elif curve_type == "MODERATE_CURVE":
        status_color = (0, 165, 255)
    elif curve_type == "GENTLE_CURVE":
        status_color = (0, 255, 255)
    else:
        status_color = (0, 255, 0)

    cv2.putText(img, road_status, (panel1_x + 140, panel1_y + 111),
               font, font_scale, status_color, font_thickness)

    # Panel 2: Speed & Safety Panel (Bottom Right)
    panel2_width = 200
    panel2_height = 80
    panel2_x = img_width - panel2_width - 15
    panel2_y = img_height - panel2_height - 15

    # Create semi-transparent panel background
    overlay2 = img.copy()
    cv2.rectangle(overlay2, (panel2_x, panel2_y), (panel2_x + panel2_width, panel2_y + panel2_height),
                 (0, 0, 0), -1)
    cv2.addWeighted(overlay2, 0.7, img, 0.3, 0, img)

    # Panel border
    cv2.rectangle(img, (panel2_x, panel2_y), (panel2_x + panel2_width, panel2_y + panel2_height),
                 (255, 255, 255), 2)

    # Panel title
    cv2.rectangle(img, (panel2_x, panel2_y), (panel2_x + panel2_width, panel2_y + 25),
                 (0, 150, 0), -1)
    cv2.putText(img, "SAFETY STATUS", (panel2_x + 10, panel2_y + 18),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Safety indicators
    if lane_curve < 300:
        safety_text = "CAUTION"
        safety_color = (0, 165, 255)
    elif vehicle_count > 0:
        safety_text = "MONITOR"
        safety_color = (0, 255, 255)
    else:
        safety_text = "CLEAR"
        safety_color = (0, 255, 0)

    cv2.putText(img, safety_text, (panel2_x + 10, panel2_y + 50),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, safety_color, 2)

    return img

# Step 8: Main execution with better error handling
def main():
    """Main function to run the enhanced lane detection"""
    # Run calibration
    print("Running camera calibration...")
    undistort_img()

    # Test YOLO model with a sample
    print("Testing YOLO model...")
    if yolo_model is not None:
        try:
            # Create a test image
            test_img = np.ones((480, 640, 3), dtype=np.uint8) * 128
            test_vehicles = detect_vehicles(test_img)
            print(f"YOLO model test: {len(test_vehicles)} vehicles detected in test image")
        except Exception as e:
            print(f"YOLO test failed: {e}")
    else:
        print("Warning: YOLO model not loaded properly")

    # Check if video exists
    video_path = 'project_video.mp4'
    if not os.path.exists(video_path):
        # Try alternative names
        alternative_paths = ['project_video.mov', 'video.mp4', 'test_video.mp4', 'input.mp4']
        found = False
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                video_path = alt_path
                found = True
                print(f"Found video: {video_path}")
                break

        if not found:
            print(f"Video file not found. Tried: {video_path}, {', '.join(alternative_paths)}")
            print("Please upload your video file and update the video_path variable.")
            return

    # Process video
    print("Processing video...")
    try:
        myclip = VideoFileClip(video_path)
        output_vid = 'enhanced_lane_detection_output.mp4'

        print(f"Video duration: {myclip.duration:.2f} seconds")
        print(f"Video size: {myclip.size}")
        print(f"Video FPS: {myclip.fps}")

        # Process with higher quality settings
        clip = myclip.fl_image(vid_pipeline)
        clip.write_videofile(output_vid,
                           audio=False,
                           codec='libx264',
                           temp_audiofile='temp-audio.m4a',
                           remove_temp=True,
                           verbose=False,
                           logger=None)

        print(f"Video processing complete! Output saved as: {output_vid}")

        # Display video
        return HTML(f"""
        <video width="1280" height="720" controls>
          <source src="{output_vid}" type="video/mp4">
        </video>
        """)

    except Exception as e:
        print(f"Video processing error: {e}")
        print("Make sure your video file exists and is accessible.")
        import traceback
        traceback.print_exc()

# Run the main function
if __name__ == "__main__":
    result = main()
    if result:
        display(result)

Initializing YOLOv5...
Ultralytics installed successfully
Custom model not found, using YOLOv5s pretrained...


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-7-26 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLOv5 loaded on cuda
Running camera calibration...
Camera calibration successful: 1.7494785357235567
Testing YOLO model...
YOLO model test: 0 vehicles detected in test image
Processing video...
Video duration: 50.40 seconds
Video size: [1280, 720]
Video FPS: 25.0
Video processing complete! Output saved as: enhanced_lane_detection_output.mp4
